In [ ]:
# Make sure that you have all these libaries available to run the code successfully
import quandl
#from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.metrics import mean_squared_error

import pytz

In [ ]:
# Here we only get the data in S&P 500
universe_df = pd.read_csv("sp500.csv")
universe = universe_df['Symbol'].tolist()
end_date = '2018-03-27'
begin_date = pd.Timestamp(end_date) - pd.DateOffset(months=24)
def get_stock_data(ticker):

    data_source = 'Yahoo Finance' # alphavantage or kaggle
    if data_source == 'Yahoo Finance':
        # ====================== Loading Data from Yahoo Finance ===========================
        
        df = pd.read_csv("SP500_constitutes_modified.csv")
        df = df[["Date", ticker]]
        
    else:
        # ====================== Loading Data from Quandl ==================================

        api_key = 'tv6mxJKZxRcTysJaHKc2'

        # Save data to this file
        file_to_save = 'stock_market_data-%s.csv'%ticker
        quandl.ApiConfig.api_key = api_key
         # If you haven't already saved data,
        # Go ahead and grab the data from the url
        # And store date, low, high, volume, close, open values to a Pandas DataFrame
        if not os.path.exists(file_to_save):
            df = quandl.get_table('WIKI/PRICES', qopts = { 'columns': ['date', 'close'] }, ticker = [ticker], date = { 'gte': '2016-03-27', 'lte': '2018-03-27' })        
            df.to_csv(file_to_save)
            
            # If the data is already there, just load it from the CSV
        else:
            print('File already exists. Loading data from CSV')
            df = pd.read_csv(file_to_save)
    
    return df

In [ ]:
# Here we only get the data in S&P 500
universe_df = pd.read_csv("sp500.csv")
universe = universe_df['Symbol'].tolist()
end_date = '2018-03-27'
begin_date = pd.Timestamp(end_date) - pd.DateOffset(months=24)
def get_stock_data(ticker):

    data_source = 'Yahoo Finance' # alphavantage or kaggle
    if data_source == 'Yahoo Finance':
        df = pd.read_csv("SP500_constitutes_modified.csv")
        df = df[["Date", ticker]]
        
    elif data_source == 'quantdl':
        # ====================== Loading Data from Quandl ==================================

        api_key = 'tv6mxJKZxRcTysJaHKc2'

        # Save data to this file
        file_to_save = 'stock_market_data-%s.csv'%ticker
        quandl.ApiConfig.api_key = api_key
         # If you haven't already saved data,
        # Go ahead and grab the data from the url
        # And store date, low, high, volume, close, open values to a Pandas DataFrame
        if not os.path.exists(file_to_save):
            df = quandl.get_table('WIKI/PRICES', qopts = { 'columns': ['date', 'close'] }, ticker = [ticker], date = { 'gte': '2016-03-27', 'lte': '2018-03-27' })        
            df.to_csv(file_to_save)
            
            # If the data is already there, just load it from the CSV
        else:
            print('File already exists. Loading data from CSV')
            df = pd.read_csv(file_to_save)
    else:
    
        # ====================== Loading Data from Kaggle ==================================
        # You will be using HP's data. Feel free to experiment with other data.
        # But while doing so, be careful to have a large enough dataset and also pay attention to the data normalization
        df = pd.read_csv(os.path.join('Stocks','hpq.us.txt'),delimiter=',',usecols=['Date','Open','High','Low','Close'])
        print('Loaded data from the Kaggle repository')
    
    return df

In [ ]:
testing_pair = ['CME-US', 'ICE-US']
df_1 = get_stock_data('CME-US')
df_2 = get_stock_data('ICE-US')

In [ ]:
close1 = df_1[testing_pair[0]].tolist()
close2 = df_2[testing_pair[1]].tolist()
date_all = df_1.Date.tolist()
spread_close = [s1 - s2 for s1,s2 in zip(close1,close2)]
spread_df = pd.DataFrame(spread_close, index = date_all, columns = ['close']) 
spread_df

In [ ]:
def create_dataset(dataset, look_back=1):

    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        dataX.append(dataset[i:(i+look_back),0])
        dataY.append(dataset[i+look_back,0])

    return np.array(dataX), np.array(dataY)


def OLS_model(data):

    #load data
    dataset = np.array(data['close'].values).reshape(-1,1)
    dataset = dataset.astype('float32')

    scaler = MinMaxScaler(feature_range=(0,1))
    dataset = scaler.fit_transform(dataset)

    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    # reshape for look_back
    look_back = 10
    X_train, y_train = create_dataset(train, look_back)
    X_test, y_test = create_dataset(test, look_back)

    # reshape for LSTM [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # LSTM
    model = Sequential()
    model.add(LSTM(32, input_dim=1)) #look_back))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, nb_epoch=20, batch_size=5, verbose=2)

    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test) 
    
    # scale back 
    train_pred = scaler.inverse_transform(train_pred)
    y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
    test_pred = scaler.inverse_transform(test_pred)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # shift predictions for plotting
    train_pred_plot = np.empty_like(dataset)
    train_pred_plot[:,:] = np.nan
    train_pred_plot[look_back:len(train_pred)+look_back,:] = train_pred

    test_pred_plot = np.empty_like(dataset)
    test_pred_plot[:,:] = np.nan
    test_pred_plot[len(train_pred)+(look_back*2)+1:len(dataset)-1,:] = test_pred

    f = plt.figure()
    plt.plot(scaler.inverse_transform(dataset), color='b', lw=2.0, label='S&P 500')
    plt.plot(train_pred_plot, color='g', lw=2.0, label='LSTM train')
    plt.plot(test_pred_plot, color='r', lw=2.0, label='LSTM test')
    plt.legend(loc=3)
    plt.grid(True)
    plt.show()
    return y_train, train_pred, y_test, test_pred

In [ ]:
y_train, train_pred, y_test, test_pred = OLS_model(spread_df)